# Get the uniprot GO terms of molecular function ready for modelling

In [6]:
# Gene Ontology can be found here: http://geneontology.org/page/ontology-documentation
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
from collections import defaultdict

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD

import re
from bioservices import *
import collections
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [7]:
train = pd.read_csv('..//bases/new_train_variants.csv')
test = pd.read_csv('..//bases/new_test_variants.csv')

In [8]:
data_all = pd.concat((train, test), axis=0, ignore_index=True)

In [9]:
all_genes = set(data_all.Gene)
print(len(all_genes))
print(all_genes)

401
{'SF3B1', 'MAP2K1', 'EIF1AX', 'SLC33A1', 'WHSC1', 'STK11', 'ITM2B', 'CSF1R', 'RAD54L', 'BRCA1', 'EPHA5', 'GNA11', 'FLT3', 'PRKRA', 'EWSR1', 'RXRA', 'PPP2R1A', 'CIC', 'LCT', 'NSD1', 'NPM1', 'MAP2K2', 'CHST3', 'ATRX', 'AP3B1', 'BCL2', 'JAK2', 'SUCLA2', 'TP53BP1', 'MYD88', 'ACVR1', 'TSC1', 'SLC6A5', 'ROS1', 'GNAS', 'TRPM1', 'JAK1', 'ALG10', 'RICTOR', 'LRP5', 'BTK', 'CCND3', 'MOCS1', 'IDH2', 'VHL', 'MYOT', 'SPAST', 'CLDN16', 'KMT2C', 'PIK3CD', 'DNMT3A', 'FBXW7', 'TNFRSF11A', 'CCND1', 'AKAP9', 'EIF2B5', 'TINF2', 'MGA', 'SLC7A9', 'ARID2', 'CHEK2', 'WHSC1L1', 'CDKN2A', 'MPL', 'FAM58A', 'IKZF1', 'EPHB2', 'SRC', 'MOCS2', 'AURKB', 'SCN4A', 'PPM1D', 'ATM', 'TGFBR1', 'RBM10', 'CARD11', 'FLNB', 'KMT2D', 'LRP4', 'ARID1A', 'APOL1', 'SHOC2', 'FLT1', 'RET', 'SOX17', 'POLE', 'HLA-A', 'KDM6A', 'BBS5', 'ATP2C1', 'FUBP1', 'NTRK2', 'TGM5', 'HNF1A', 'RAB27A', 'RBBP8', 'B4GALT7', 'GLE1', 'CDKN1A', 'RIT1', 'NTRK3', 'BMPR1B', 'CDK6', 'TTK', 'CDKN1B', 'SMAD3', 'CTLA4', 'ZFPM2', 'MAP3K1', 'NF1', 'SCO2', 'MYCN

In [10]:
u = UniProt()

In [11]:
u.debugLevel = "INFO"
u.timeout = 100   # some queries are long and requires much more time; default is 1000 seconds

In [12]:
gene_entry_dict = {}
class_dict = {}
for gene in all_genes:
    keyword = 'gene:%s+AND+organism:9606' %gene #to query database, with gene and organism 9606 is Homo Sapien (human)
    entry_name_tab = u.search(keyword, frmt='tab', limit=1, columns="entry name") 
    entry_name = [s.strip() for s in entry_name_tab.splitlines()][1] # gets the entry name = in second position in list
    gene_entry_dict[gene] = entry_name

In [13]:
gene_entries = list(gene_entry_dict.values())
len(gene_entries)

401

In [14]:
df = u.get_df(gene_entries) # searches in uniprot -> gets results back 
df

INFO:root:fetching information from uniprot for 399 entries
INFO:root:uniprot.get_df 1/3
INFO:root:uniprot.get_df 2/3
INFO:root:uniprot.get_df 3/3
INFO:root:uniprot.get_df 4/3


Entry   Entry name                                     Gene names  \
0    P27986   P85A_HUMAN                                  [PIK3R1 GRB1]   
1    Q12809  KCNH2_HUMAN                          [KCNH2 ERG ERG1 HERG]   
2    Q12888  TP53B_HUMAN                                      [TP53BP1]   
3    P11362  FGFR1_HUMAN         [FGFR1 BFGFR CEK FGFBR FLG FLT2 HBGFR]   
4    P12830  CADH1_HUMAN                                [CDH1 CDHE UVO]   
5    P17948  VGFR1_HUMAN                          [FLT1 FLT FRT VEGFR1]   
6    P84022  SMAD3_HUMAN                                  [SMAD3 MADH3]   
7    P38936  CDN1A_HUMAN  [CDKN1A CAP20 CDKN1 CIP1 MDA6 PIC1 SDI1 WAF1]   
8    P63092  GNAS2_HUMAN                               [GNAS GNAS1 GSP]   
9    Q15303  ERBB4_HUMAN                                   [ERBB4 HER4]   
10   O75581   LRP6_HUMAN                                         [LRP6]   
11   P22607  FGFR3_HUMAN                                   [FGFR3 JTK4]   
12   Q76LX8  ATS13_HUMAN             [ADAMTS13 C9orf8 UNQ6102/PRO20085]   
13   P82279  CRUM1_HUMAN                                         [CRB1]   
14   O96017   CHK2_HUMAN                        [CHEK2 CDS1 CHK2 RAD53]   
15   P20823  HNF1A_HUMAN                                   [HNF1A TCF1]   
16   O00329  PK3CD_HUMAN                                       [PIK3CD]   
17   Q01196  RUNX1_HUMAN                             [RUNX1 AML1 CBFA2]   
18   P51159  RB27A_HUMAN                                 [RAB27A RAB27]   
19   P49815   TSC2_HUMAN                                    [TSC2 TSC4]   
20   O75197   LRP5_HUMAN                                [LRP5 LR3 LRP7]   
21   O95255   MRP6_HUMAN                               [ABCC6 ARA MRP6]   
22   O15315  RA51B_HUMAN                          [RAD51B RAD51L1 REC2]   
23   Q99836  MYD88_HUMAN                                        [MYD88]   
24   Q8NCM8  DYHC2_HUMAN      [DYNC2H1 DHC1B DHC2 DNCH2 DYH1B KIAA1997]   
25   O60733  PLPL9_HUMAN                                [PLA2G6 PLPLA9]   
26   O00400  ACATN_HUMAN                            [SLC33A1 ACATN AT1]   
27   P30462   1B14_HUMAN                                   [HLA-B HLAB]   
28   P01889   1B07_HUMAN                                   [HLA-B HLAB]   
29   P30153   2AAA_HUMAN                                      [PPP2R1A]   
..      ...          ...                                            ...   
377  O43548   TGM5_HUMAN                                    [TGM5 TGMX]   
378  O15393  TMPS2_HUMAN                               [TMPRSS2 PRSS10]   
379  Q14520  HABP2_HUMAN                             [HABP2 HGFAL PHBP]   
380  Q14164   IKKE_HUMAN                     [IKBKE IKKE IKKI KIAA0151]   
381  Q9Y6J6  KCNE2_HUMAN                                        [KCNE2]   
382  P41229  KDM5C_HUMAN            [KDM5C DXS1272E JARID1C SMCX XE169]   
383  Q9UMN6  KMT2B_HUMAN      [KMT2B HRX2 KIAA0304 MLL2 MLL4 TRX2 WBP7]   
384  Q8NEZ4  KMT2C_HUMAN                     [KMT2C HALR KIAA1506 MLL3]   
385  O15151   MDM4_HUMAN                                    [MDM4 MDMX]   
386  Q9NZB8  MOCS1_HUMAN                                  [MOCS1 MIG11]   
387  P01106    MYC_HUMAN                                  [MYC BHLHE39]   
388  O15055   PER2_HUMAN                                [PER2 KIAA0347]   
389  P43116  PE2R2_HUMAN                                       [PTGER2]   
390  P09619  PGFRB_HUMAN                          [PDGFRB PDGFR PDGFR1]   
391  P35408  PE2R4_HUMAN                                [PTGER4 PTGER2]   
392  P60484   PTEN_HUMAN                              [PTEN MMAC1 TEP1]   
393  P23468  PTPRD_HUMAN                                        [PTPRD]   
394  Q9Y620  RA54B_HUMAN                                       [RAD54B]   
395  P01112   RASH_HUMAN                                   [HRAS HRAS1]   
396  P06400     RB_HUMAN                                          [RB1]   
397  Q15286  RAB35_HUMAN                              [RAB35 RAB1C RAY]   
398  Q6R327  RICTR_HUMAN       

In [15]:
df_new = df[df['Gene ontology (molecular function)'].notnull()] # don't consider genes with no molecular function

In [16]:
df_new['Gene ontology (molecular function)'] = df_new['Gene ontology (molecular function)'].apply(lambda x: x.split('; ')) #split functions based on ;


C:\Users\rafiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
GO_terms_dict = dict(zip(df_new['Entry name'], df_new['Gene ontology (molecular function)']))

In [18]:
GO_terms_dict

{'1A02_HUMAN': ['beta-2-microglobulin binding [GO:0030881]',
  'peptide antigen binding [GO:0042605]',
  'receptor binding [GO:0005102]',
  'RNA binding [GO:0003723]',
  'TAP binding [GO:0046977]',
  'T cell receptor binding [GO:0042608]'],
 '1B07_HUMAN': ['peptide antigen binding [GO:0042605]',
  'receptor binding [GO:0005102]'],
 '1B14_HUMAN': ['peptide antigen binding [GO:0042605]',
  'TAP binding [GO:0046977]'],
 '2AAA_HUMAN': ['antigen binding [GO:0003823]',
  'protein heterodimerization activity [GO:0046982]',
  'protein phosphatase regulator activity [GO:0019888]',
  'protein serine/threonine phosphatase activity [GO:0004722]'],
 'ABCBB_HUMAN': ['ATP binding [GO:0005524]',
  'bile acid-exporting ATPase activity [GO:0015432]',
  'canalicular bile acid transmembrane transporter activity [GO:0015126]',
  'sodium-exporting ATPase activity, phosphorylative mechanism [GO:0008554]',
  'transporter activity [GO:0005215]'],
 'ABL1_HUMAN': ['actin filament binding [GO:0051015]',
  'actin 

In [19]:
# Find most common GO terms to use as features
def flatten(l): # taken from https://stackoverflow.com/questions/33900770/most-frequent-values-in-a-dictionary
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, str): #replaced basestring with str for Python3
            for sub in flatten(el):
                yield sub
        else:
            yield el


In [20]:
All_GO_terms = list(flatten(GO_terms_dict.values()))
len(set(All_GO_terms))

736

In [22]:
# loading the XGboost most important 190 features
feature_scores = np.load("molecular_bases/features_molecular_function.npy")

In [23]:
features = []
for feature_score in feature_scores:
    feature = feature_score[0]
    features.append(feature)

In [24]:
len(features)

190

In [25]:
# initialize data with the features 
for feature in features:
    data_all[feature] = 0

In [27]:
# add 1 if the GO term is inside the gene_entry_dict for a particular gene
for i in data_all.index:
    gene = data_all.Gene[i]
    gene_entry = gene_entry_dict[gene]
    if gene_entry in GO_terms_dict:
        GO_terms = GO_terms_dict[gene_entry]
        features_inside = list(set(GO_terms).intersection(features))# get only features in the GO_terms that we need
        data_all.loc[i, features_inside] = 1

In [28]:
data_all.shape

(4675, 194)

In [29]:
data_all

Class    Gene   ID             Variation  \
0       1.0  FAM58A    0  Truncating Mutations   
1       2.0     CBL    1                 W802*   
2       2.0     CBL    2                 Q249E   
3       3.0     CBL    3                 N454D   
4       4.0     CBL    4                 L399V   
5       4.0     CBL    5                 V391I   
6       5.0     CBL    6                 V430M   
7       1.0     CBL    7              Deletion   
8       4.0     CBL    8                 Y371H   
9       4.0     CBL    9                 C384R   
10      4.0     CBL   10                 P395A   
11      4.0     CBL   11                 K382E   
12      4.0     CBL   12                 R420Q   
13      4.0     CBL   13                 C381A   
14      5.0     CBL   14                 P428L   
15      4.0     CBL   15                 D390Y   
16      1.0     CBL   16  Truncating Mutations   
17      4.0     CBL   17                 Q367P   
18      5.0     CBL   18                 M374V   
19      4.0     CBL   19                 Y371S   
20      6.0     CBL   20                  H94Y   
21      4.0     CBL   21                 C396R   
22      4.0     CBL   22                 G375P   
23      4.0     CBL   23                 S376F   
24      4.0     CBL   24                 P417A   
25      4.0     CBL   25                 H398Y   
26      4.0   SHOC2   26                   S2G   
27      4.0    TERT   27                 Y846C   
28      7.0    TERT   28                 C228T   
29      4.0    TERT   29                 H412Y   
...     ...     ...  ...                   ...   
4645    NaN    PTEN  957                 V133I   
4646    NaN   ABCC6  958                T1130M   
4647    NaN   CARM1  959                 S217E   
4648    NaN  EIF2B5  960                 T182M   
4649    NaN   KRIT1  961                 Q201E   
4650    NaN  PDGFRA  962     D842_I843delinsIM   
4651    NaN   SOX17  963              Deletion   
4652    NaN   CHST3  964                 F159I   
4653    NaN   FGFR1  965                 V561M   
4654    NaN    KRAS  966                  G13R   
4655    NaN    TP53  967                 K120R   
4656    NaN    GLE1  968                 I684T   
4657    NaN   ABCC6  969                D1238H   
4658    NaN    AGXT  970                  G82E   
4659    NaN    TP53  971                 S109Y   
4660    NaN    TP53  972                 R248G   
4661    NaN    TP53  973                 R280G   
4662    NaN    LRP5  974                 T253I   
4663    NaN   SEPT9  975                  S93F   
4664    NaN    SCO2  976                 S225F   
4665    NaN   GNA11  977                 R183C   
4666    NaN  RECQL4  978                R1021W   
4667    NaN    KRAS  979                  Q61P   
4668    NaN     GNE  980                 A557T   
4669    NaN   ABCC6  981                G1321S   
4670    NaN    TP63  982                 S580P   
4671    NaN   SCN4A  983                 R672G   
4672    NaN    BRAF  984                 N581H   
4673    NaN    TSHR  985                 S281N   
4674    NaN    ASS1  986                 S180N   

      magnesium ion binding [GO:0000287]  \
0                                      0   
1                                      0   
2                                      0   
3                                      0   
4                                      0   
5                                      0   
6                                      0   
7                                      0   
8                                      0   
9                                      0   
10                                     0   
11                                     0   
12                                     0   
13                                     0   
14                                     0   
15                                     0   
16                                     0   
17                                     0   
18                                     0   
19                       

In [30]:
# Save the 190 features into one csv file in case we will use it again
data_all.to_csv("molecular_bases/all_molecular_functions.csv",index=False)

In [40]:
# Do an SVD on the molecular functions to get a reduction to 25 features
svd = TruncatedSVD(n_components=25, n_iter=20, random_state=18)
feature_columns = data_all.iloc[:,4:] #starting from the 4th column we have our features
truncated_molecular = pd.DataFrame(svd.fit_transform(feature_columns.values))


In [41]:
# add truncated molecular functions to our data 
data_new = pd.concat((train, test), axis=0, ignore_index=True)
data_SVD = pd.concat((data_new, truncated_molecular), axis = 1)
data_SVD

Class    Gene   ID             Variation         0         1         2  \
0       1.0  FAM58A    0  Truncating Mutations  0.007211  0.013929 -0.001500   
1       2.0     CBL    1                 W802*  0.347313 -0.122612  0.187927   
2       2.0     CBL    2                 Q249E  0.347313 -0.122612  0.187927   
3       3.0     CBL    3                 N454D  0.347313 -0.122612  0.187927   
4       4.0     CBL    4                 L399V  0.347313 -0.122612  0.187927   
5       4.0     CBL    5                 V391I  0.347313 -0.122612  0.187927   
6       5.0     CBL    6                 V430M  0.347313 -0.122612  0.187927   
7       1.0     CBL    7              Deletion  0.347313 -0.122612  0.187927   
8       4.0     CBL    8                 Y371H  0.347313 -0.122612  0.187927   
9       4.0     CBL    9                 C384R  0.347313 -0.122612  0.187927   
10      4.0     CBL   10                 P395A  0.347313 -0.122612  0.187927   
11      4.0     CBL   11                 K382E  0.347313 -0.122612  0.187927   
12      4.0     CBL   12                 R420Q  0.347313 -0.122612  0.187927   
13      4.0     CBL   13                 C381A  0.347313 -0.122612  0.187927   
14      5.0     CBL   14                 P428L  0.347313 -0.122612  0.187927   
15      4.0     CBL   15                 D390Y  0.347313 -0.122612  0.187927   
16      1.0     CBL   16  Truncating Mutations  0.347313 -0.122612  0.187927   
17      4.0     CBL   17                 Q367P  0.347313 -0.122612  0.187927   
18      5.0     CBL   18                 M374V  0.347313 -0.122612  0.187927   
19      4.0     CBL   19                 Y371S  0.347313 -0.122612  0.187927   
20      6.0     CBL   20                  H94Y  0.347313 -0.122612  0.187927   
21      4.0     CBL   21                 C396R  0.347313 -0.122612  0.187927   
22      4.0     CBL   22                 G375P  0.347313 -0.122612  0.187927   
23      4.0     CBL   23                 S376F  0.347313 -0.122612  0.187927   
24      4.0     CBL   24                 P417A  0.347313 -0.122612  0.187927   
25      4.0     CBL   25                 H398Y  0.347313 -0.122612  0.187927   
26      4.0   SHOC2   26                   S2G  0.206740  0.058513 -0.087834   
27      4.0    TERT   27                 Y846C  0.492162 -0.208593  0.336370   
28      7.0    TERT   28                 C228T  0.492162 -0.208593  0.336370   
29      4.0    TERT   29                 H412Y  0.492162 -0.208593  0.336370   
...     ...     ...  ...                   ...       ...       ...       ...   
4645    NaN    PTEN  957                 V133I  0.623422  0.001956  0.288702   
4646    NaN   ABCC6  958                T1130M  0.326728  0.327783 -0.057115   
4647    NaN   CARM1  959                 S217E  0.365566 -0.236827  0.437756   
4648    NaN  EIF2B5  960                 T182M  0.000000 -0.000000  0.000000   
4649    NaN   KRIT1  961                 Q201E  0.016019  0.029919  0.003990   
4650    NaN  PDGFRA  962     D842_I843delinsIM  0.933815  1.530852  0.316078   
4651    NaN   SOX17  963              Deletion  0.805174 -0.523673 -0.014021   
4652    NaN   CHST3  964                 F159I  0.000000 -0.000000  0.000000   
4653    NaN   FGFR1  965                 V561M  1.131618  1.550301  0.245464   
4654    NaN    KRAS  966                  G13R  0.020258  0.034674  0.002577   
4655    NaN    TP53  967                 K120R  4.432643 -1.922303 -1.696754   
4656    NaN    GLE1  968                 I684T  0.246412  0.066543 -0.087902   
4657    NaN   ABCC6  969                D1238H  0.326728  0.327783 -0.057115   
4658    NaN    AGXT  970                  G82E  0.205908  0.063182 -0.103308   
4659    NaN    TP53  971                 S109Y  4.432643 -1.922303 -1.696754   
4660    NaN    TP53  972                 R248G  4.432643 -1.922303 -1.696754   
4661    NaN    TP53  973                 R280G  4.432643 -1.922303 -1.696754   
4662    NaN    LRP5  974                 T253I  0.000091  0.000190 -0.000009   
4663  

In [44]:
print(svd.explained_variance_ratio_.sum())

0.816023021311


In [57]:
new_names = [] 
for i in range(25):
    new_names.append('molecular_SVD_'+str(i+1))

data_SVD.columns = data_SVD.columns[:4].tolist() + new_names

In [46]:
# Save the 25 svd's features into one file 
data_SVD.to_csv("molecular_bases/svd25_molecular_functions.csv",index=False)